# decision tree

In [22]:
import os
import re
import pandas as pd
import seaborn as sns
from Bio import *
import matplotlib.pyplot as plt
import warnings
import numpy as np
from Bio import SeqIO
import os
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix


here, we are required to merge the bigbigdf from the snippy analysis to the phenotype of the phages on 1 defence system. 

In [3]:
def find_key_gene_db_name(filtered_intergenic, startstopdict, position):
    found = 0
    for key in startstopdict.keys(): 
        array = startstopdict[key]
        if position in array: 
            found = 1
            protein_annotation = filtered_intergenic.loc[key, 'protein annotation']
            db_name = filtered_intergenic.loc[key, 'db name']
    if found == 0: 
        protein_annotation = 'no gene'
        db_name = 'no gene'
    return protein_annotation, db_name

def filter_snippy_df(morphology_reference, morphology_reads, reference_phage, inter_or_intra): 
    myo_snippy_df = snippy_df[(snippy_df.loc[:, 'morphology reference'] == morphology_reference) & (snippy_df.loc[:, 'morphology read'] == morphology_reads)]
    pa10myo_snippy_df = myo_snippy_df[myo_snippy_df.loc[:, 'reference'] == reference_phage]
    pa10myo_snippy_df = pa10myo_snippy_df[pa10myo_snippy_df.loc[:, 'inter or intra'] == inter_or_intra]
    return pa10myo_snippy_df

def filter_intergenic_df(phage): 
    df = gbk_df[gbk_df.loc[:, 'phage'] == phage]
    return df

In [4]:
snippy_df = pd.read_csv(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\final_snippy_analysis'
           + '/' + 'snippy_df.csv')
gbk_df = pd.read_csv(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\compressed dataframes' + '/' + 
                    "intergenic_information_df.csv")
phenotype_df = pd.read_csv(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\make phenotypic array' + 
                          '/' + 'binary_scaled_myo_plate_reader_df.csv').set_index('phage')

In [5]:
# dictionary = {}
# start_stop_big_array = []
# for ii in range(gbk_df.shape[0]): 
#     start = gbk_df.loc[ii, 'start']
#     stop = gbk_df.loc[ii, 'stop']
#     dictionary[ii] = np.arange(start, stop + 1, 1)
    
# gbk_df.loc[:, 'start stop pos array'] = dictionary.values()

# #from this point on, we filter further, on the myos and specifically Pa10. 
# phage = 'Pa12'
# filtered_snippy = filter_snippy_df('Myoviridae', 'Myoviridae', phage, 'inter').reset_index(drop = True)
# filtered_intergenic = filter_intergenic_df(phage = phage)
# startstopdict = dict(filtered_intergenic.loc[:, 'start stop pos array'])

# #here, I merge the dataset with the gene the snp finds itself in
# for ii in range(filtered_snippy.shape[0]): 
#     position = filtered_snippy.loc[ii, 'POS']
#     protein_annotation, db_name = find_key_gene_db_name(filtered_intergenic, startstopdict, position)
#     filtered_snippy.loc[ii, 'protein annotation'] = protein_annotation
#     filtered_snippy.loc[ii, 'db name'] = db_name

In [133]:
file_list_snippy_files = os.listdir(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\compressed dataframes\filtered_snippy_files')
all_filtered_snippy = []
for filtered_snippy_file in file_list_snippy_files: 
    filtered_snippy = pd.read_csv(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\compressed dataframes\filtered_snippy_files' 
                                 + '/' + filtered_snippy_file)
    filtered_snippy.loc[:, 'phage'] = [filtered_snippy_file[:-4]]*filtered_snippy.shape[0]
    all_filtered_snippy = all_filtered_snippy + [filtered_snippy]
all_snippy = pd.concat(all_filtered_snippy)

In [134]:
all_snippy

,Unnamed: 0,CHROM,POS,ID,REF,ALT,QUAl,family,inter or intra,reference,reads,phage_x,morphology reference,phage_y,morphology read,protein annotation,db name,phage
0,0,40848+(circular),226,.,C,T,390.256,0,inter,Pa10,Pa12,Pa10,Myoviridae,Pa12,Myoviridae,no gene,no gene,Pa10
1,1,40848+(circular),287,.,C,A,1004.770,0,inter,Pa10,Pa12,Pa10,Myoviridae,Pa12,Myoviridae,Phage protein,['SEED:fig|287.19287.peg.2'],Pa10
2,2,40848+(circular),305,.,TGAC,CGAT,1175.090,0,inter,Pa10,Pa12,Pa10,Myoviridae,Pa12,Myoviridae,Phage protein,['SEED:fig|287.19287.peg.2'],Pa10
3,3,40848+(circular),331,.,ACA,CTG,1144.870,0,inter,Pa10,Pa12,Pa10,Myoviridae,Pa12,Myoviridae,Phage protein,['SEED:fig|287.19287.peg.2'],Pa10
4,4,40848+(circular),350,.,C,G,1481.740,0,inter,Pa10,Pa12,Pa10,Myoviridae,Pa12,Myoviridae,Phage protein,['SEED:fig|287.19287.peg.2'],Pa10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10612,10612,61662+(circular),65486,.,G,A,1639.000,0,inter,Pa49,Pa10,Pa49,Myoviridae,Pa10,Myoviridae,Phage protein,['SEED:fig|287.19304.peg.92'],Pa49
10613,10613,61662+(circular),65495,.,GGCG,AGCA,1586.710,0,inter,Pa49,Pa10,Pa49,Myoviridae,Pa10,Myoviridae,Phage protein,['SEED:fig|287.19304.peg.92'],Pa49
10614,10614,61662+(circular),65554,.,T,C,1586.190,0,inter,Pa49,Pa10,Pa49,Myoviridae,Pa10,Myoviridae,Phage protein,['SEED:fig|287.19304.peg.92'],Pa49
10615,10615,61662+(circular),65591,.,T,G,1712.520,0,inter,Pa49,Pa10,Pa49,Myoviridae,Pa10,Myoviridae,Phage protein,['SEED:fig|287.19304.peg.92'],Pa49


In [132]:
ref_all_snippy = all_snippy[all_snippy.loc[:, 'reference'] == 'Pa10']
ds_ref_all_snippy = pd.merge(ref_all_snippy, phenotype_df.loc[:, 'Zorya I'].reset_index(), left_on='reads', right_on='phage')
x_df = ds_ref_all_snippy[['POS', 'inter or intra', 'protein annotation', 'db name']]
dummy_x_df = pd.get_dummies(x_df)
dummy_x_df.loc[:, 'reads'] = list(ds_phenotype_df.loc[:, 'reads'])
#ds_ref_all_snippy.loc[:, 'reads']

C:\Users\Halewijn\AppData\Local\Temp/ipykernel_19148/2794071930.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'phage_x'} in the result is deprecated and will raise a MergeError in a future version.
  ds_ref_all_snippy = pd.merge(ref_all_snippy, phenotype_df.loc[:, 'Zorya I'].reset_index(), left_on='reads', right_on='phage')


KeyError: 'reads'